In [96]:
import numpy as np
import os
from pathlib import Path
from collections import OrderedDict
from os import makedirs

In [100]:
windowsizes = [5,10,15,20,30,50]
replay_dir = Path("../dataset")

In [101]:
for windowsize in windowsizes:
    replay_list = []

    for replay_name in os.listdir(replay_dir / 'dataset'):
            if replay_name.endswith('.npz'):
                replay = np.load(replay_dir / 'dataset' / replay_name, allow_pickle=True)
                # replay.keys()
                replay_dict = OrderedDict(replay)



                ### only use pics close to the reward

                images = replay_dict['image']
                wood = replay_dict['inventory_wood']
                
                ## gather indices where a new woodlog is gathered
                reward_ix = []
                current_wood = 0
                
                for i,w in enumerate(wood):
                    if w > current_wood:
                        reward_ix.append(i)
                        current_wood = w
                    if w < current_wood:
                        current_wood = w
                        
                ## get cleaned data
                for key in replay_dict.keys():

                    cleaned =  [replay_dict[key][max(0,i-windowsize):i] for i in reward_ix]

                    if len(cleaned[0].shape) > 1: 
                        replay_dict[key] = np.vstack([replay_dict[key][max(0,i-windowsize):i] for i in reward_ix])
                    else:
                        replay_dict[key] = np.concatenate([replay_dict[key][max(0,i-windowsize):i] for i in reward_ix])
                
                ## save to new folders
                replay_list.append(replay_dict)
                makedirs(replay_dir / f'dataset_windowsize={windowsize}',exist_ok=True)
                np.savez_compressed(replay_dir / f'dataset_windowsize={windowsize}' / replay_name,**replay_dict,)